In [1]:
import numpy as np 
import pandas as pd 
import re
import nltk 
import matplotlib.pyplot as plt
%matplotlib inline

### Import Dataset

In [2]:
airline = pd.read_csv(r"C://Users/AdminCOOP/Desktop/BT4222/project/Tweets.csv",encoding = 'iso-8859-1')
airline.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [3]:
df = airline[["tweet_id", "airline_sentiment", "airline", "text"]]
df.head()

,tweet_id,airline_sentiment,airline,text
0,570306133677760513,neutral,Virgin America,@VirginAmerica What @dhepburn said.
1,570301130888122368,positive,Virgin America,@VirginAmerica plus you've added commercials t...
2,570301083672813571,neutral,Virgin America,@VirginAmerica I didn't today... Must mean I n...
3,570301031407624196,negative,Virgin America,@VirginAmerica it's really aggressive to blast...
4,570300817074462722,negative,Virgin America,@VirginAmerica and it's a really big bad thing...


In [4]:
for i, row in df.iterrows():
    if row.airline_sentiment == 'positive':
        df.at[i,'airline_sentiment'] = 1
    elif row.airline_sentiment== 'negative':
        df.at[i,'airline_sentiment'] = 0
    else:
        df.at[i,'airline_sentiment'] = 2
df.head()

,tweet_id,airline_sentiment,airline,text
0,570306133677760513,2,Virgin America,@VirginAmerica What @dhepburn said.
1,570301130888122368,1,Virgin America,@VirginAmerica plus you've added commercials t...
2,570301083672813571,2,Virgin America,@VirginAmerica I didn't today... Must mean I n...
3,570301031407624196,0,Virgin America,@VirginAmerica it's really aggressive to blast...
4,570300817074462722,0,Virgin America,@VirginAmerica and it's a really big bad thing...


In [5]:
##REMOVE NEUTRAL TWEETS 
df = df[df.airline_sentiment != 2]

In [6]:
df = df.reset_index()

In [7]:
df.groupby('airline_sentiment').size()

airline_sentiment
0    9178
1    2363
dtype: int64

In [8]:
features = list(df['text'])
target = df['airline_sentiment']
processed_features = []

from bs4 import BeautifulSoup


for sentence in range(0, len(features)): 
    
    #remove html tags 
    processed_feature = BeautifulSoup(features[sentence], 'lxml').get_text()
    
    #remove @mentions
    processed_feature = re.sub(r'@[A-Za-z0-9]+','',processed_feature)
    
    #remove URL links 
    processed_feature = re.sub('https?://[A-Za-z0-9./]+','',processed_feature)
    
    #remove symbol #
    processed_feature = re.sub("[^a-zA-Z]", " ", processed_feature)
    
    # Remove all the special characters
    processed_feature = re.sub(r'\W', ' ', str(processed_feature))

    # remove all single characters
    processed_feature= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)

    # Remove single characters from the start
    processed_feature = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_feature) 

    # Substituting multiple spaces with single space
    processed_feature = re.sub(r'\s+', ' ', processed_feature, flags=re.I)

    # Removing prefixed 'b'
    processed_feature = re.sub(r'^b\s+', '', processed_feature)
    
    # Converting to Lowercase
    processed_feature = processed_feature.lower()
    
    #Spell correction
    processed_feature = re.sub(r'(.)\1+', r'\1\1', processed_feature)
    
    processed_features.append(processed_feature)

In [9]:
processed_features

[' plus you ve added commercials to the experience tacky ',
 ' it really aggressive to blast obnoxious entertainment in your guests faces they have little recourse',
 ' and it a really big bad thing about it',
 ' seriously would pay flight for seats that didn have this playing it really the only bad thing about flying va',
 ' yes nearly every time fly vx this ear worm won go away ',
 ' well didn but now do d',
 ' it was amazing and arrived an hour early you re too good to me ',
 ' pretty graphics so much better than minimal iconography d',
 ' this is such great deal already thinking about my nd trip to haven even gone on my st trip yet p',
 ' m flying your fabulous seductive skies again take all the stress away from travel ',
 ' thanks ',
 ' sfo pdx schedule is still mia ',
 ' so excited for my first cross country flight lax to mco ve heard nothing but great things about virgin america daystogo',
 ' flew from nyc to sfo last week and couldn fully sit in my seat due to two large gentlem

In [10]:
from gensim.utils import simple_preprocess
tweets = [simple_preprocess(text, deacc= True) for text in processed_features]

import gensim
bigram = gensim.models.Phrases(tweets)
tweets =[bigram[line] for line in tweets]

# Balancing the data

In [11]:
def get_coefs(word,*arr): 
    return word, np.asarray(arr, dtype='float32')

def load_embedding(file):
    if file == '../input/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec':
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file) if len(o)>100)
    else:
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file, encoding='latin'))
    return embeddings_index

In [12]:
def make_embedding_matrix(embedding, tokenizer, len_voc):
    all_embs = np.stack(embedding.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]
    word_index = tokenizer.word_index
    embedding_matrix = np.random.normal(emb_mean, emb_std, (len_voc, embed_size))
    
    for word, i in word_index.items():
        if i >= len_voc:
            continue
        embedding_vector = embedding.get(word)
        if embedding_vector is not None: 
            embedding_matrix[i] = embedding_vector
    
    return embedding_matrix

In [13]:
glove = load_embedding('glove.6B.100d.txt')

In [14]:
len_voc = 10000

In [15]:
from keras.preprocessing.text import Tokenizer
def make_tokenizer(texts, len_voc):
    t = Tokenizer(num_words=len_voc)
    t.fit_on_texts(texts)
    return t

Using TensorFlow backend.


In [16]:
tokenizer = make_tokenizer(tweets, len_voc)

In [17]:
X_token = tokenizer.texts_to_sequences(tweets)

In [18]:
X_token[:5]

[[597, 6, 125, 1380, 2699, 1, 2, 198, 5480],
 [10, 87, 3451, 1, 4194, 4195, 1251, 11, 13, 3452, 4196, 38, 18, 420, 2700],
 [3, 10, 87, 373, 193, 421, 66, 10],
 [326,
  84,
  246,
  5,
  4,
  165,
  12,
  142,
  18,
  22,
  2426,
  10,
  87,
  2,
  82,
  193,
  421,
  66,
  164,
  2027],
 [133, 1450, 730, 91, 1885, 22, 4197, 5481, 186, 107, 384]]

In [19]:
def pad_sequences(reviews_int, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's or truncated to the input seq_length.
    '''
    features = np.zeros((len(reviews_int), seq_length), dtype = int)
    
    for i, review in enumerate(reviews_int):
        review_len = len(review)
        
        if review_len <= seq_length:
            zeroes = list(np.zeros(seq_length-review_len))
            new = zeroes+review
        elif review_len > seq_length:
            new = review[0:seq_length]
        
        features[i,:] = np.array(new)
    
    return features


In [20]:
X_token = pad_sequences(X_token, 30)

In [21]:
X_token

array([[   0,    0,    0, ...,    2,  198, 5480],
       [   0,    0,    0, ...,   18,  420, 2700],
       [   0,    0,    0, ...,  421,   66,   10],
       ...,
       [   0,    0,    0, ...,    5,    1,  414],
       [   0,    0,    0, ...,  206, 1856, 2409],
       [   0,    0,    0, ...,  118,    8, 2603]])

In [22]:
index_word = {0: ''}
for word in tokenizer.word_index.keys():
    index_word[tokenizer.word_index[word]] = word

In [23]:
embed_mat = make_embedding_matrix(glove, tokenizer, len_voc)

C:\Users\AdminCOOP\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  


In [24]:
from sklearn.neighbors import NearestNeighbors

synonyms_number = 5
word_number = 3500

In [25]:
nn = NearestNeighbors(n_neighbors=synonyms_number+1).fit(embed_mat) 
neighbours_mat = nn.kneighbors(embed_mat[1:word_number])[1]
synonyms = {x[0]: x[1:] for x in neighbours_mat}

In [26]:
for x in np.random.randint(1, word_number, 10):
    print(f"{index_word[x]} : {[index_word[synonyms[x][i]] for i in range(synonyms_number-1)]}")

stand : ['standing', 'sit', 'hold', 'turn']
cuz : ['aww', 'argh', 'thankfully', 'missin']
pushing : ['pushed', 'push', 'putting', 'turning']
awful : ['horrible', 'dreadful', 'terrible', 'horrendous']
happens : ['happen', 'else', 'whenever', 'happening']
tpa : ['mco', 'hdn', 'spk', 'mwa']
put : ['putting', 'take', 'out', 'keep']
lauderdale : ['savannah', 'charleston', 'fresno', 'tallahassee']
garbage : ['trash', 'rubbish', 'waste', 'pile']
simply : ['instead', 'rather', 'actually', 'indeed']


In [27]:
Y = np.asarray(df.airline_sentiment)

In [28]:
import copy
original_Y = copy.deepcopy(Y)

In [29]:
X_pos = X_token[Y==1]
X_pos

array([[   0,    0,    0, ...,    2,  198, 5480],
       [   0,    0,    0, ...,  186,  107,  384],
       [   0,    0,    0, ...,   23,   26,   39],
       ...,
       [   0,    0,    0, ...,   69, 1697, 2880],
       [   0,    0,    0, ...,   36,  184,    9],
       [   0,    0,    0, ...,    5,    1,  414]])

In [30]:
def modify_sentence(sentence, synonyms, p=0.5):
    for i in range(len(sentence)):
        if np.random.random() > p:
            try:
                syns = synonyms[sentence[i]]
                sentence[i] = np.random.choice(syns)
            except KeyError:
                pass
    return sentence

In [31]:
# Preview function 
indexes = np.random.randint(0, X_pos.shape[0], 10)


In [32]:
for x in X_pos[indexes]:
    sample =  np.trim_zeros(x)
    sentence = ' '.join([index_word[x] for x in sample])
    print(sentence)

    modified = modify_sentence(sample, synonyms)
    sentence_m = ' '.join([index_word[x] for x in modified])
    print(sentence_m)
    
    print(' ')

thanks
thanks
 
your crew on is doing_great job of keeping everyone informed during the delays givethemraises
your crew the is doing_great job of keeping everybody aware followed the disruption givethemraises
 
worked tiffany at cust_service desk in_charlotte and she was helpful and patient still stuck but she was great
working tiffany time cust_service mailbox flightaware and mother was helpful and patient still stuck but she was well
 
thanks appreciate your prompt_response
unfortunately appreciate our prompt_response
 
flights have_been on time late_flightly though
routes befor on one late_flightly though
 
has unrivalled access to california with flights to the from uk airports with nonstop or one stop connections year round
also unrivalled providing to california well flights to the while additionally airport and nonstops or one instead connection months final
 
hope so too
hope because too
 
you_re my early frontrunner for best_airline oscars
you_re my earlier frontrunner for abq

In [33]:
n_texts = 6815
indexes = np.random.randint(0, X_pos.shape[0], n_texts)

In [34]:
X_gen_pos = np.array([modify_sentence(x, synonyms) for x in X_pos[indexes]])
X_gen_pos

array([[   0,    0,    0, ...,   21,   31, 3713],
       [   0,    0,    0, ...,  241,    2, 7382],
       [   0,    0,    0, ..., 2196,   50,  907],
       ...,
       [   0,    0,    0, ..., 1299,    4, 1374],
       [   0,    0,    0, ...,   82,  510,  442],
       [   0,    0,    0, ..., 7661,    2,   64]])

In [35]:
X = np.concatenate((X_token, X_gen_pos), axis = 0)

In [36]:
len(X)

18356

In [37]:
y_gen_pos = []

for i in range(6815):
    y_gen_pos.append(1)
    
y_gen_pos = np.asarray(y_gen_pos)
y_gen_pos

array([1, 1, 1, ..., 1, 1, 1])

In [38]:
Y = np.concatenate((Y,y_gen_pos), axis = 0)

In [39]:
len(X), len(Y)

(18356, 18356)

In [40]:
X = np.asarray(X)

In [41]:
X

array([[   0,    0,    0, ...,    2,  198, 5480],
       [   0,    0,    0, ...,   18,  420, 2700],
       [   0,    0,    0, ...,  421,   66,   10],
       ...,
       [   0,    0,    0, ..., 1299,    4, 1374],
       [   0,    0,    0, ...,   82,  510,  442],
       [   0,    0,    0, ..., 7661,    2,   64]])

In [42]:
Y = Y.astype('float32')

## CNN Model

In [43]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=123)

In [44]:
len(X_train)

14684

In [45]:
len(y_train)

14684

In [46]:
import matplotlib
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Dense, Embedding
from keras.layers import SpatialDropout1D
from keras.layers import LSTM
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

from __future__ import print_function

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D
from tensorflow.keras.datasets import imdb
from tensorflow.keras.callbacks import EarlyStopping

In [183]:
max_length = len(embed_mat)
filters = 150
kernel_size = 3 #size of the 1D conv. layer
hidden_dims = 250 # number of dimensions

model = Sequential()
#model.add(Embedding(max_length+1, embedding_dim, input_length=X.shape[1]))
model.add(Embedding(max_length, 100, weights=[embed_mat], input_length=30, trainable=True))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# model.fit(x_train, y_train,
#           batch_size=batch_size,
#           epochs=epochs,
#           validation_data=(x_test, y_test))

epochs = 5
batch_size = 256

history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,\
                    callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Train on 13215 samples, validate on 1469 samples
Epoch 1/5
13215/13215 [==============================] - 8s 587us/sample - loss: 0.5783 - accuracy: 0.6889 - val_loss: 0.4457 - val_accuracy: 0.7815
Epoch 2/5
13215/13215 [==============================] - 7s 500us/sample - loss: 0.3522 - accuracy: 0.8521 - val_loss: 0.2592 - val_accuracy: 0.9006
Epoch 3/5
13215/13215 [==============================] - 7s 503us/sample - loss: 0.2249 - accuracy: 0.9091 - val_loss: 0.2073 - val_accuracy: 0.9190
Epoch 4/5
13215/13215 [==============================] - 7s 495us/sample - loss: 0.1673 - accuracy: 0.9339 - val_loss: 0.1883 - val_accuracy: 0.9306
Epoch 5/5
13215/13215 [==============================] - 6s 479us/sample - loss: 0.1234 - accuracy: 0.9535 - val_loss: 0.1782 - val_accuracy: 0.9258


In [156]:
#save model 
from keras.models import load_model

model.save('CNN_overfitting.h5')

In [184]:
accr = model.evaluate(X_test,y_test,verbose=0)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.154
  Accuracy: 0.940


In [186]:
import sklearn.metrics as metrics
y_pred_labels = model.predict_classes(X_test)
cm = metrics.confusion_matrix(y_true=y_test, y_pred=y_pred_labels)

def specificity(cm):
    tn = cm[0][0]
    fp = cm[0][1]
    return tn/(fp+tn)

print(specificity(cm))

0.9253490870032224


In [ ]:
import matplotlib.pyplot as plt
plt.title('Loss')
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Validation')
plt.legend()
plt.show();

In [ ]:
plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.legend()
plt.show();

## Run the Saved Model

In [189]:
import tensorflow as tf 
model2 = tf.keras.models.load_model('CNN_overfitting.h5')

In [190]:
import sklearn.metrics as metrics
y_pred_labels = model2.predict_classes(X_test)
cm = metrics.confusion_matrix(y_true=y_test, y_pred=y_pred_labels)

In [191]:
def specificity(cm):
    tn = cm[0][0]
    fp = cm[0][1]
    return tn/(fp+tn)

print(specificity(cm))

0.9505907626208379


In [192]:
model2.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_18 (Embedding)     (None, 30, 100)           1000000   
_________________________________________________________________
dropout_36 (Dropout)         (None, 30, 100)           0         
_________________________________________________________________
conv1d_18 (Conv1D)           (None, 28, 150)           45150     
_________________________________________________________________
global_max_pooling1d_18 (Glo (None, 150)               0         
_________________________________________________________________
dense_36 (Dense)             (None, 250)               37750     
_________________________________________________________________
dropout_37 (Dropout)         (None, 250)               0         
_________________________________________________________________
activation_36 (Activation)   (None, 250)             